In [1]:
import math
import random
import pandas as pd
import numpy as np
import json
import scipy.special as scs
import ast

In [23]:

def compute_hourly_risk(uni_density,
                        norm_density,
                        skewed_density,
                        people_displacement_within_hour,
                        mdt):
    '''
    An hour can be comprised of 12 sections (5 min interval).
    Each section has their own number of people.
    Each section is multiplied by the densities, sampled twice and computes their risk.

    uni_density: an array of size 'num_hexagons'. Refer to uniform distribution
    norm_density: an array of size 'num_hexagons'. Refer to normal distribution
    skewed_density,: an array of size 'num_hexagons'. Refer to skewed distribution
    people_displacement_within_hour: a dictionary comprised of 12 sections (5 min interval)
                                     and hold the number of people within that section

    mdt: mediam dwell time in that POI
    '''

    # N - Total number of people within the POI in the whole hour
    N = sum(people_displacement_within_hour.values())

    # set the M to be the number of infected individuals in that POI
    T = math.floor(N/3)
    if T != 0:
        dist = [T - i for i in range(T)]
        M = random.choices(range(0, T), dist)
        M = M[0]
    else:
        dist = 0
        M = 0

    uni_risk = 0
    norm_risk = 0
    skewed_risk = 0
    for key, occup in people_displacement_within_hour.items():

        # calculate unifrom densities
        uni_dist_indices_chosen = random.choices(range(len(uni_density)), uni_density, k=occup)
        uni_dist = [0]*len(uni_density)

        for ind in uni_dist_indices_chosen:
            uni_dist[ind] += 1

            # calculate normal densities
        norm_dist_indices_chosen = random.choices(range(len(norm_density)), norm_density, k=occup)
        norm_dist = [0]*len(norm_density)

        for ind in norm_dist_indices_chosen:
            norm_dist[ind] += 1

            # calculate skewed densities
        skewed_dist_indices_chosen = random.choices(range(len(skewed_density)), skewed_density, k=occup)
        skewed_dist = [0]*len(skewed_density)

        for ind in skewed_dist_indices_chosen:
            skewed_dist[ind] += 1

            # calculate uniform risk
        uni_hourly_risk = random.choices(uni_dist,
                                         [1/len(uni_density)]*len(uni_density),
                                         k=2)
        for h in uni_hourly_risk:
            if N-h >= M:
                uni_risk += 1.0 - (scs.comb(N-h,M) / scs.comb(N,M))

        # calculate normal risk
        norm_hourly_risk = random.choices(norm_dist,
                                          [1/len(norm_density)]*len(norm_density),
                                          k=2)
        for h in norm_hourly_risk:
            if N-h >= M:
                norm_risk += 1.0 - (scs.comb(N-h,M) / scs.comb(N,M))

        # calculate skewed risk
        skewed_hourly_risk = random.choices(skewed_dist,
                                            [1/len(skewed_density)]*len(skewed_density),
                                            k=2)
        for h in skewed_hourly_risk:
            if N-h >= M:
                skewed_risk += 1.0 - (scs.comb(N-h,M) / scs.comb(N,M))

    uni_risk_of_infection = 1-(1-(uni_risk / 120.0))**mdt
    uni_risk_of_infection = 0 if uni_risk_of_infection is None else uni_risk_of_infection
    
    normal_risk_of_infection = 1-(1-(norm_risk / 120.0))**mdt
    normal_risk_of_infection = 0 if normal_risk_of_infection is None else normal_risk_of_infection
    
    skewed_risk_of_infection = 1-(1-(skewed_risk / 120.0))**mdt
    skewed_risk_of_infection = 0 if skewed_risk_of_infection is None else skewed_risk_of_infection

    return uni_risk_of_infection, normal_risk_of_infection, skewed_risk_of_infection


def compute_hourly_risks(uni_density,
                         norm_density,
                         skewed_density,
                         people_displacement_within_hours,
                         mdt):
    '''
    uni_density: an array of size 'num_hexagons'. Refer to uniform distribution
    norm_density: an array of size 'num_hexagons'. Refer to normal distribution
    skewed_density,: an array of size 'num_hexagons'. Refer to skewed distribution
    people_displacement_within_hours: an array of dictionaries, each comprises of 12 sections (5 min interval)
                                     that hold the number of people within that section

    mdt: mediam dwell time in that POI
    mdt: mediam dwell time in that POI
    '''
    uni_risks = []
    norm_risks = []
    skewed_risks = []

    for i in range(len(people_displacement_within_hours)):
        uni_risk, norm_risk, skewed_risk = compute_hourly_risk(uni_density,
                                                               norm_density,
                                                               skewed_density,
                                                               people_displacement_within_hours[i],
                                                               mdt)

        uni_risks.append(uni_risk)
        norm_risks.append(norm_risk)
        skewed_risks.append(skewed_risk)

    return uni_risks, norm_risks, skewed_risks


def get_poi_occupancy_within_an_hour(json_string, num_people_in_that_hour):
    '''
    returns a dictionary that specifies of how many people are within a specific time slot of a 5 min increment
    '''
    dwell_time = json.loads(json_string)

    # assigns each person a dwell time based on dwell time distribution
    sum_people = dwell_time['<5'] + dwell_time['5-10'] + dwell_time['11-20'] + dwell_time['21-60'] + dwell_time['61-120'] + dwell_time['121-240'] + dwell_time['>240']

    people_dist_dwell = random.choices([
        2.5,
        7.5,
        15.5,
        40.0
    ],
        [dwell_time['<5'] / sum_people,
         dwell_time['5-10'] / sum_people,
         dwell_time['11-20'] / sum_people,
         (dwell_time['21-60'] + dwell_time['61-120'] + dwell_time['121-240'] + dwell_time['>240'])/sum_people
         ], k=int(float(num_people_in_that_hour))
    )

    # assign a time of arrival for each person
    people_arrival = random.choices([
        0, 5, 10, 15, 20, 25, 30,
        35, 40, 45, 50, 55
    ], k=int(num_people_in_that_hour))

    # record results
    result = {"0-4": 0,
              "5-9": 0,
              "10-14": 0,
              "15-19": 0,
              "20-24": 0,
              "25-29": 0,
              "30-34": 0,
              "35-39": 0,
              "40-44": 0,
              "45-49": 0,
              "50-54": 0,
              "55-59": 0,
              }

    for i in range(int(num_people_in_that_hour)):
        if 0 <= people_arrival[i] < 4 or \
                0 <= people_arrival[i]+people_dist_dwell[i] < 5:
            result["0-4"] +=1
        elif 5 <= people_arrival[i] < 10 or \
                5 <= people_arrival[i]+people_dist_dwell[i] < 10:
            result["5-9"] +=1
        elif 10 <= people_arrival[i] < 15 or \
                10 <= people_arrival[i]+people_dist_dwell[i] < 15:
            result["10-14"] +=1
        elif 15 <= people_arrival[i] < 20 or \
                15 <= people_arrival[i]+people_dist_dwell[i] < 20:
            result["15-19"] +=1
        elif 20 <= people_arrival[i] < 25 or \
                20 <= people_arrival[i]+people_dist_dwell[i] < 25:
            result["20-24"] +=1
        elif 25 <= people_arrival[i] < 30 or \
                25 <= people_arrival[i]+people_dist_dwell[i] < 30:
            result["25-29"] +=1
        elif 30 <= people_arrival[i] < 35 or \
                30 <= people_arrival[i]+people_dist_dwell[i] < 35:
            result["30-34"] +=1
        elif 35 <= people_arrival[i] < 40 or \
                35 <= people_arrival[i]+people_dist_dwell[i] < 40:
            result["35-39"] +=1
        elif 40 <= people_arrival[i] < 45 or \
                40 <= people_arrival[i]+people_dist_dwell[i] < 45:
            result["40-44"] +=1
        elif 45 <= people_arrival[i] < 50 or \
                45 <= people_arrival[i]+people_dist_dwell[i] < 50:
            result["45-49"] +=1
        elif 50 <= people_arrival[i] < 55 or \
                50 <= people_arrival[i]+people_dist_dwell[i] < 55:
            result["50-54"] +=1
        else:
            result["55-59"] +=1

    return result


def get_all_hourly_occupancies(json_string, num_people_in_that_hours):
    '''
    returns an array or dictionaries that specifies of how many people are within a specific time slot of a 5 min increment.
    Each entry refers to one hour of a week
    '''
    results = []

    for i in range(len(num_people_in_that_hours)):
        results.append(get_poi_occupancy_within_an_hour(json_string, num_people_in_that_hours[i]))

    return results


In [6]:
# tmp_file_for_risk_processing.csv was computer from POI-Explorer.ipynb.

tmp = pd.read_csv("tmp_file_for_risk_processing.csv")
tmp = tmp.drop(columns=['visits_by_each_hour',
                        'poi_cbg', 'cum_sum',
                        'ext_visitor_upper',
                        'ext_visits_upper',
                        'normalized_hourly_visits',
                        'total_visits'])

# calculate the number of hexagons in each POI
HEX_SIZE = 4 # m^2

tmp['num_hexagons'] = [math.ceil(float(x)/HEX_SIZE) for x in tmp['geodesic_area']]

# set uniform densities
tmp['uniform_densities'] = [[1 / x]*x for x in tmp['num_hexagons']]

# assign random values for each hexagom for each POI
occup = [random.choices(range(0, 30), k=x) for x in tmp['num_hexagons']]
sum_of_occup = [sum(x) for x in occup]

# set skewed densities
tmp['skewed_densities'] = [[float(j / sum_of_occup[i]) for j in occup[i]] for i in range(len(sum_of_occup))]

# set normal densities
std_occup = [np.std(x) for x in occup]
mean_occup = [np.mean(x) for x in occup]
norm_prob_density = [
    [(1 / (std_occup[i] * np.sqrt(2*np.pi))) * np.exp(-0.5*((j-mean_occup[i])/std_occup[i])**2)
     for j in occup[i]]
    for i in range(len(occup))
]

sum_of_occup = [sum(x) for x in norm_prob_density]
tmp['normal_densities'] = [[float(j / sum_of_occup[i]) for j in norm_prob_density[i]] for i in range(len(sum_of_occup))]


In [20]:


tmp['num_people_within_hour'] = [get_all_hourly_occupancies(tmp['bucketed_dwell_times'][x], ast.literal_eval(tmp['true_hourly_visits'][x])) for x in range(len(tmp))]



In [57]:
risks = [compute_hourly_risks(tmp['uniform_densities'][x],
                              tmp['normal_densities'][x],
                              tmp['skewed_densities'][x],
                              tmp['num_people_within_hour'][x],
                              tmp['median_dwell'][x])
         for x in range(len(tmp))]

<ipython-input-23-d92252561dc5>:65: RuntimeWarning: invalid value encountered in double_scalars
  uni_risk += 1.0 - (scs.comb(N-h,M) / scs.comb(N,M))
<ipython-input-23-d92252561dc5>:73: RuntimeWarning: invalid value encountered in double_scalars
  norm_risk += 1.0 - (scs.comb(N-h,M) / scs.comb(N,M))
<ipython-input-23-d92252561dc5>:81: RuntimeWarning: invalid value encountered in double_scalars
  skewed_risk += 1.0 - (scs.comb(N-h,M) / scs.comb(N,M))


In [67]:
risks = np.array(risks)
uniform_risks = risks[:,0]
normal_risks = risks[:,1]
skewed_risks = risks[:,2]



In [68]:
len(skewed_risks[:,167])

7970

In [69]:
for i in range(168):
    tmp['uniform_risks_'+str(i)] = uniform_risks[:,i]
    tmp['normal_risks'+str(i)] = normal_risks[:,i]
    tmp['skewed_risks'+str(i)] = skewed_risks[:,i]

# tmp['uniform_risks'] = uniform_risks
# tmp['normal_risks'] = normal_risks
# tmp['skewed_risks'] = skewed_risks

tmp.to_csv('GTA_tmp_file_for_risk_processing_with_densities.csv')


In [18]:
old_risks = pd.read_csv('ca_poi_risks_2021-04-19-one-week.csv')
# tmp = pd.read_csv('GTA_tmp_file_for_risk_processing_with_densities.csv')
print(len(tmp))
tmp = pd.concat([tmp, old_risks[['placekey', 'safegraph_place_id', 'top_category']]], axis=1, join="inner")
print(len(tmp))


7970
7970


In [19]:
tmp.head(10)

,Unnamed: 0,Unnamed: 0.1,location_name,true_hourly_visits,tup_key,safegraph_place_id,median_dwell,geodesic_area,latitude,longitude,...,skewed_risks165,uniform_risks_166,normal_risks166,skewed_risks166,uniform_risks_167,normal_risks167,skewed_risks167,placekey,safegraph_place_id,top_category
0,0,0,Anytime Fitness,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Anytime Fitness', '35212150')",sg:093da07111ba4a6fbf87953e5b6cf451,19.0,565.875254,43.784431,-79.661567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,zzw-222@65x-9m6-qfz,sg:006a0dff74f34238b25186af152aebb8,Specialized Freight Trucking
1,1,1,Nobleton Custom Woodwork,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Nobleton Custom Woodwork', '35190261')",sg:0c0499a6e1dd43edaab91be9e03e78c3,41.0,12258.350719,43.811377,-79.531200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22d-222@3wx-yd9-c5z,sg:03eef9a4da2f4fa1a062dc3e4228ae72,Restaurants and Other Eating Places
2,2,2,Tim Hortons,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Tim Hortons', '35204144')",sg:26c988c03a5f4cea8df251ca4c86d145,8.5,67.564867,43.796095,-79.423339,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,222-222@66b-2dy-ht9,sg:04b12c3a6eba4a6c8b40a1c1cdba1243,Building Equipment Contractors
3,3,3,Kingshall Realty Services,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Kingshall Realty Services', '35204315')",sg:2e928e4342154d48bbfd1b4405135694,9.0,121.393601,43.773286,-79.421472,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,zzw-222@5sz-yq2-zxq,sg:0556e5faae5449429835ce62a6071bbd,Grocery Stores
4,4,4,Fabric House,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('Fabric House', '35204746')",sg:5893d0e33eb84f6097c6fc8dcc1d38f9,15.0,11437.709948,43.806796,-79.221656,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,222-222@665-zky-p7q,sg:0605aff4eae14a4bb50b84f634ca7384,"Promoters of Performing Arts, Sports, and Simi..."
5,5,5,UPS Customer Centre,"[111.28742019884878, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('UPS Customer Centre', '35190261')",sg:70892bcbba3a4566881a5d50491b9dde,40.0,42787.726661,43.777337,-79.517749,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,222-222@63t-x38-3bk,sg:08b4ffd6edc9430c8b3c6fe44fa2c796,"Automotive Parts, Accessories, and Tire Stores"
6,6,6,A & W Pharmacy,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('A & W Pharmacy', '35200094')",sg:747c69b7538c4fe593e1a4f0f7ad7703,340.5,440.969174,43.815256,-79.295337,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,zzw-222@665-v48-h5z,sg:09e4857fe58f446f9e4c34fe17864b4a,Motion Picture and Video Industries
7,7,7,The Suya Spot Scarborough,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('The Suya Spot Scarborough', '35203660')",sg:9c65d5a612b749bea7d1bbe7beb17d7b,51.0,56.319782,43.771571,-79.186929,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,227-222@3x6-rzh-8qf,sg:0c0c7c98904f42748cb0cbdc4c05fb71,Health and Personal Care Stores
8,8,8,DryShield Water Solutions,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","('DryShield Water Solutions', '35190261')",sg:a8152d75236f4b3c8d274d5b4c85fde0,61.0,4789.575485,43.803427,-79.528003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,222-222@5x6-ptn-h3q,sg:0c3154809d134c9cb4377c980f4df24d,Amusement Parks and Arcades
9,9,9,Jack's Taekwon DO Fitness Centre,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(""Jack's Taekwon DO Fitness Centre"", '35204725')",sg:ae2c3b7a4d764e178eda27ea7f0d3ed2,64.0,1161.402520,43.802594,-79.198280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,223-222@65x-945-835,sg:0e41de78ce1f483f8b545ea109e2a97a,Nursing Care Facilities (Skilled Nursing Facil...


In [20]:
tmp = tmp.drop(columns=['Unnamed: 0',
                        'Unnamed: 0.1',
                        'tup_key'])
tmp.head(2)

,location_name,true_hourly_visits,safegraph_place_id,median_dwell,geodesic_area,latitude,longitude,bucketed_dwell_times,num_hexagons,uniform_densities,...,skewed_risks165,uniform_risks_166,normal_risks166,skewed_risks166,uniform_risks_167,normal_risks167,skewed_risks167,placekey,safegraph_place_id,top_category
0,Anytime Fitness,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",sg:093da07111ba4a6fbf87953e5b6cf451,19.0,565.875254,43.784431,-79.661567,"{""<5"":0,""5-10"":8,""11-20"":5,""21-60"":4,""61-120"":...",142,"[0.007042253521126761, 0.007042253521126761, 0...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,zzw-222@65x-9m6-qfz,sg:006a0dff74f34238b25186af152aebb8,Specialized Freight Trucking
1,Nobleton Custom Woodwork,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",sg:0c0499a6e1dd43edaab91be9e03e78c3,41.0,12258.350719,43.811377,-79.531200,"{""<5"":0,""5-10"":0,""11-20"":0,""21-60"":1,""61-120"":...",3065,"[0.0003262642740619902, 0.0003262642740619902,...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22d-222@3wx-yd9-c5z,sg:03eef9a4da2f4fa1a062dc3e4228ae72,Restaurants and Other Eating Places


In [21]:
# This file would be used in evaluation.ipynb for further processing tradeoffs
# of different policies and how they affect the risk factor.
# It would also be used in poi_near_me.py to calculate the risk at a poi at a given time

tmp.to_csv('GTA_risks.csv', index=False)
